In [37]:
import pandas as pd
import numpy as np

artist_data = pd.read_csv('datasets/ydata-ymusic-user-artist-rating.csv')
artist_data = artist_data.loc[artist_data['rating'] <= 100]
artist_data.sort_values('rating', ascending=False)

,user_id,artist_id,rating
1048574,17484,1010722,100
263778,4346,1043477,100
446869,7519,1006971,100
100111,1675,1012152,100
446870,7519,1007098,100
...,...,...,...
839286,14081,1004599,0
608438,10439,1024496,0
839284,14081,1004561,0
839283,14081,1004546,0


In [38]:
#preparing data table for analysis
user_rating = pd.pivot_table(data=artist_data, values='rating', index='user_id', columns='artist_id')
user_rating.head()

artist_id,24538,1000004,1000006,1000010,1000012,1000015,1000016,1000018,1000021,1000023,...,1101383,1101399,1101400,1101401,1101416,1101471,1101613,1101630,1101671,1101719
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
#calculating mean for every artist
avg_rating = pd.DataFrame(artist_data.groupby('artist_id')['rating'].mean())
avg_rating

,rating
artist_id,
24538,90.000000
1000004,44.731183
1000006,56.207692
1000010,52.000000
1000012,43.884956
...,...
1101471,5.555556
1101613,90.000000
1101630,7.454545


In [40]:
avg_rating.sort_values('rating', ascending = False).head(20)

,rating
artist_id,
1069861,100.0
1054044,100.0
1031121,100.0
1009432,100.0
1054050,100.0
1017988,100.0
1098250,100.0
1009455,100.0
1031090,100.0


In [56]:
#artist_id = 1005698
#see artist 1058037 rated by user
artist = 1058037
artist_user_rating = user_rating[artist]
artist_user_rating.head()

user_id
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
Name: 1058037, dtype: float64

In [42]:
#applying correlation for the entire user rating dataframe
#Collaborative Filtering Song Recommendation
#Pearson Correlation
recommended_artist = user_rating.corrwith(artist_user_rating)
recommended_artist

artist_id
24538     NaN
1000004   NaN
1000006   NaN
1000010   NaN
1000012   NaN
           ..
1101471   NaN
1101613   NaN
1101630   NaN
1101671   NaN
1101719   NaN
Length: 14147, dtype: float64

In [43]:
#removing nan values
recommended_artist_corr = pd.DataFrame(recommended_artist, columns=['Correlation'])
recommended_artist_corr.dropna(inplace=True)

In [23]:
recommended_artist_corr

,Correlation,rating
artist_id,,
1000264,0.469095,53.290120
1000335,1.000000,62.321682
1000362,0.775796,26.795455
1000656,1.000000,55.336683
1000903,1.000000,25.672414
...,...,...
1098588,1.000000,61.409752
1098640,1.000000,45.907143
1098924,1.000000,31.000000


In [47]:
#sort the recommended artist correlation
recommended_artist_corr=recommended_artist_corr.sort_values('Correlation', ascending=False)
recommended_artist_corr

,Correlation,rating
artist_id,,
1098306,1.0,45.210526
1008776,1.0,41.386364
1098033,1.0,16.190476
1019053,1.0,49.080645
1059036,1.0,74.750000


In [48]:
recommended_artist_corr.sort_values('rating', ascending=False).head()

,Correlation,rating
artist_id,,
1059036,1.0,74.750000
1019053,1.0,49.080645
1098306,1.0,45.210526
1008776,1.0,41.386364
1098033,1.0,16.190476


In [51]:
recommended_artist_corr.columns

Index(['Correlation', 'rating'], dtype='object')

In [ ]:
#filtering still need to be done